In [7]:
# !pip3 install git+https://github.com/rongardF/tvdatafeed.git@e6f6aaa7de439ac6e454d9b26d2760ded8dc4923
from tvDatafeed import TvDatafeed, Interval

In [8]:
from helpers_db import get_engine, run_sql
engine = get_engine()

In [16]:
from helpers_queries import queries
assets = None
with engine.begin() as conn:
  assets = run_sql(conn, queries['list_assets_longterm_bars']).mappings().all()
assets

[{'symbol': 'AAOI', 'exchange': 'NASDAQ', 'bars_w': 200, 'bars_d': 1332},
 {'symbol': 'AAPL', 'exchange': 'NASDAQ', 'bars_w': 126, 'bars_d': 814},
 {'symbol': 'ALB', 'exchange': 'NYSE', 'bars_w': 219, 'bars_d': 1465},
 {'symbol': 'AMZN', 'exchange': 'NASDAQ', 'bars_w': 94, 'bars_d': 590},
 {'symbol': 'BABA', 'exchange': 'NYSE', 'bars_w': 100, 'bars_d': 631},
 {'symbol': 'CI', 'exchange': 'NYSE', 'bars_w': 131, 'bars_d': 849},
 {'symbol': 'CLS', 'exchange': 'NYSE', 'bars_w': 221, 'bars_d': 1479},
 {'symbol': 'CRDF', 'exchange': 'NASDAQ', 'bars_w': 48, 'bars_d': 268},
 {'symbol': 'CSCO', 'exchange': 'NASDAQ', 'bars_w': 220, 'bars_d': 1472},
 {'symbol': 'DBK', 'exchange': 'XETR', 'bars_w': 121, 'bars_d': 779},
 {'symbol': 'ESEA', 'exchange': 'NASDAQ', 'bars_w': 141, 'bars_d': 919},
 {'symbol': 'GCT', 'exchange': 'NASDAQ', 'bars_w': 93, 'bars_d': 583},
 {'symbol': 'GOOG', 'exchange': 'NASDAQ', 'bars_w': 122, 'bars_d': 786},
 {'symbol': 'INMD', 'exchange': 'NASDAQ', 'bars_w': 237, 'bars_d':

In [17]:
from helpers_db import insert_on_conflict_nothing_ticks, truncate_table

tv = TvDatafeed()

def fetch_ticks(symbol, exchange, interval=Interval.in_daily, bars=100):
  df = tv.get_hist(
      symbol=symbol,
      exchange=exchange,
      interval=interval,
      n_bars=bars,
      extended_session=True,
  )
  return df

def fix_symbol(eachange_symbol):
  return eachange_symbol.split(':')[-1] if ':' in eachange_symbol else eachange_symbol

def normalize_dataframe(df):
  df.reset_index(inplace=True)
  df['symbol'] = df['symbol'].apply(fix_symbol)
  if 'datetime' in list(df.columns.values):
    df['dt'] = df['datetime']
    df.drop(columns=['datetime'], inplace=True)
  return df

for asset in assets:
  symbol    = asset['symbol']
  exchange  = asset['exchange']
  print(f"Processing {exchange}:{symbol}")
  try:
    bars      = asset['bars_d']
    interval  = Interval.in_daily
    df        = fetch_ticks(symbol=symbol, exchange=exchange, bars=bars, interval=interval) # 7*24
    df        = normalize_dataframe(df)
    df.to_sql('ticks_1d', engine, if_exists='append', index=False, method=insert_on_conflict_nothing_ticks)
    # df.to_sql('ticks_1w', engine, if_exists='append', index=False, method=insert_on_conflict_nothing_ticks)
    print(f"  count: {df.shape[0]}")
  except Exception as e:
    print(f"Exception {exchange}:{symbol}")
    print(str(e))


Processing NASDAQ:AAOI
  count: 200
Processing NASDAQ:AAPL
  count: 126
Processing NYSE:ALB
  count: 219
Processing NASDAQ:AMZN
  count: 94
Processing NYSE:BABA
  count: 100
Processing NYSE:CI
  count: 131
Processing NYSE:CLS
  count: 221
Processing NASDAQ:CRDF
  count: 48
Processing NASDAQ:CSCO
  count: 220
Processing XETR:DBK
  count: 121
Processing NASDAQ:ESEA
  count: 141
Processing NASDAQ:GCT
  count: 85
Processing NASDAQ:GOOG
  count: 122
Processing NASDAQ:INMD
  count: 237
Processing NYSE:JPM
  count: 136
Processing NYSE:KO
  count: 110
Processing EURONEXT:LQQ
  count: 132
Processing NASDAQ:META
  count: 220
Processing NASDAQ:MSFT
  count: 132
Processing NASDAQ:NKTX
  count: 112
Processing NASDAQ:NVDA
  count: 132
Processing NASDAQ:ON
  count: 126
Processing NYSE:PAYC
  count: 135
Processing NASDAQ:PYPL
  count: 94
Processing NASDAQ:SBUX
  count: 150
Processing NASDAQ:SEDG
  count: 132
Processing NYSE:T
  count: 237
Processing NASDAQ:WDAY
  count: 133
Processing NYSE:HEI
  count